# Augmenting LLMs with Retrieval, Tools, and Long-term Memory

In [1]:
!pip install google-generativeai faiss-cpu

In [2]:
from google.colab import userdata
import os

google_api_key = userdata.get('GOOGLE_API_KEY')

In [20]:
import google.generativeai as genai

genai.configure(api_key=google_api_key)

## Retrieval Augmented Generation

In [4]:
with open('knowledge_base.txt', 'r', encoding='utf-8') as file:
    file_content = file.read()

In [7]:
documents = [para.strip() for para in file_content.split('\n\n') if para.strip()]

In [8]:
print(f'The knowledge base contains {len(documents)} document.')
print(f'Example: {documents[0]}')

The knowledge base contains 10 document.
Example: In a groundbreaking discovery in education, researchers at the fictitious Aurora Institute have unveiled a method that transforms traditional classrooms into interactive learning hubs using augmented reality. This discovery promises to reshape how knowledge is delivered.


In [11]:
embeddings = []
for doc in documents:
    response = genai.embed_content(
        model="models/text-embedding-004",
        content=doc,
        task_type="retrieval_document"
    )
    embeddings.append(response['embedding'])

In [12]:
import faiss

dimension = len(embeddings[0])
index = faiss.IndexFlatL2(dimension)

In [13]:
import numpy as np

embeddings_array = np.array(embeddings).astype('float32')
index.add(embeddings_array)

In [20]:
def query_faiss_index(user_query, top_k=1):
    response = genai.embed_content(
        model="models/text-embedding-004",
        content=user_query,
        task_type="retrieval_document"
    )
    query_embedding = np.array(response['embedding']).astype('float32').reshape(1, -1)
    distances, indices = index.search(query_embedding, top_k)
    relevant_document = [documents[idx].strip() for idx in indices[0] if idx != -1]

    return relevant_document[0]

In [29]:
def retrieval_augmented_generation(user_query):
    relevant_document = query_faiss_index(user_query)
    system_prompt = f"""
    You are a knowledgeable assistant specializing in education who gives well-grounded and informative responses to users' queries. If the user's query is not relevant to the context, say 'I don't have much information about that' but give them directions, please don't mention 'based on the context', answer directly.
    """
    prompt = f"""
                  Answer the following user's query:
                  {user_query}

                  By relying on the following context:
                  {relevant_document}
              """
    model = genai.GenerativeModel(model_name="gemini-1.5-flash", system_instruction=system_prompt)
    response = model.generate_content(prompt)
    return response.text


In [30]:
print(retrieval_augmented_generation("What does EdTech startup Lumina Learning suggest?"))

Lumina Learning suggests a gamified curriculum that turns regular lessons into engaging quests.  This approach aims to boost student motivation (by 70%, according to their claims), and also develop teamwork and problem-solving abilities.



In [32]:
print(retrieval_augmented_generation("What do you know about InfinitGraph?"))

I don't have much information about InfinitGraph.  To find information about InfinitGraph, I suggest you try searching online using a search engine like Google, Bing, or DuckDuckGo.  You can also try searching on specialized technology websites or forums.



## Tools

In [27]:
def get_book_catalog(category: str):
    """
    List all books in the specified category along with their links.

    Args:
        category (str): The book category. Acceptable values are "Islamic Books" or "Self-Development Books".

    Returns:
        list[dict]: A list of dictionaries, each containing the title and link of a book.
    """
    if "islam" in category.lower():
        return [
            {"title": "Understanding Islam", "link": "https://www.islam.gov.qa/BinZaid/lang-ebook/pdf/Understanding-Islam-English.pdf"},
            {"title": "The Prophet of Islam ﷺ Muhammad", "link": "https://drive.google.com/file/d/1lb4xqzmexSNuxjX_oRHvkHuJdLCum3vd/view?usp=sharing"},
            {"title": "Muhammad ﷺ in the Bible", "link": "https://drive.google.com/file/d/1lUS5tRwHaFLueUyWKWCRR5wLpguTsVf_/view?usp=sharing"}
        ]
    elif category.lower() == "self-development books":
        return [
            {"title": "Atomic Habits", "link": "https://jamesclear.com/atomic-habits"},
        ]
    else:
        return ["No books available yet."]

functions = {
    "get_book_catalog": get_book_catalog,
}

In [28]:
model = genai.GenerativeModel(
    model_name="gemini-1.5-flash", tools=functions.values()
)

chat = model.start_chat(enable_automatic_function_calling=True)

In [29]:
response = chat.send_message(
    "Can you give me some books to learn more about islam? Please provide the links."
)
print(response.text)

Here are some books about Islam:

*   **Understanding Islam**: [https://www.islam.gov.qa/BinZaid/lang-ebook/pdf/Understanding-Islam-English.pdf](https://www.islam.gov.qa/BinZaid/lang-ebook/pdf/Understanding-Islam-English.pdf)
*   **The Prophet of Islam ﷺ Muhammad**: [https://drive.google.com/file/d/1lb4xqzmexSNuxjX_oRHvkHuJdLCum3vd/view?usp=sharing](https://drive.google.com/file/d/1lb4xqzmexSNuxjX_oRHvkHuJdLCum3vd/view?usp=sharing)
*   **Muhammad ﷺ in the Bible**: [https://drive.google.com/file/d/1lUS5tRwHaFLueUyWKWCRR5wLpguTsVf_/view?usp=sharing](https://drive.google.com/file/d/1lUS5tRwHaFLueUyWKWCRR5wLpguTsVf_/view?usp=sharing)

Please note that the availability of these links might change over time.

